In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import time
import math

In [2]:
def objfun(x,y):
    
    """ 
    This function calculates the objective function value corresponding to x and y

    Parameters: 
        x (float): x-value of candidate solution
        y (float): y-value of candidate solution

    Returns: 
        f (float): objective function value for f1 and f2: [f1,f2]
    """
    
    f = -(y + 47) * np.sin(np.sqrt(np.abs(y + x/2 + 47))) - x * np.sin(np.sqrt(np.abs(x - (y + 47))))
    
    return f

In [40]:
def check_feasibility(z):
    
    """ 
    This function checks whether the candidate solution is feasible according to given constraint

    Parameters: 
        z (float): x-value or y-value of candidate solution

    Returns: 
        feasibility_flag (bool): The flag that indicates whether the candidate solution is feasible. 
                                 If it is, then feasibility_flag = True, else False
    """
    
    # Set constraint values
    constr_ceiling = 512
    const_floor = -512
    
    # Check whether the candidate solution is valid subject to problem constraint, return True is yes and False if no
    if z <= constr_ceiling and z >= const_floor:
        
        feasibility_flag = True
        
    else:
        
        feasibility_flag = False
    
    return feasibility_flag   

In [17]:
def dataframe_init(n):
    
    """ 
    This function initialises a dataframe according to the number of particles passed based
    on a 2-variable objective function, including initial values for swarm locations and velocities
    
    The dataframe holds as columns (for each particle) the current values of x,y and f, 
    the best values of x,y, and f as well as the velocity and randomly generated rho1 and rho2

    Parameters: 
        n (int): number of particles

    Returns: 
        data_store (DataFrame): Dataframe outlined in the function description
        global_best (list): List of new global best solution in the form [x,y,f]
    """
    # Initialise list to store column names and global best solutions 
    # Make initial f infinite so that it is guaranteed to be improved upon
    columns_list = []
    global_best = [0, 0, np.inf]
    
    # Create list of column names
    for i in range(n):
    
        columns_list.append("x_"+str(i))
        columns_list.append("y_"+str(i))
        columns_list.append("f_"+str(i))
        columns_list.append("x_best_"+str(i))
        columns_list.append("y_best_"+str(i))
        columns_list.append("f_best_"+str(i))
        columns_list.append("vx_"+str(i))
        columns_list.append("vy_"+str(i))
        columns_list.append("rho1_"+str(i))
        columns_list.append("rho2_"+str(i))
    
    # Initialise dataframe with all column names
    data_store = pd.DataFrame(columns = columns_list)
    
    # Randomly initialise swarm locations, velocities and rho and store in dataframe
    for j in range(n):
        
        # Locations
        x_init = random.uniform(-512, 512)
        y_init = random.uniform(-512, 512)
        data_store.loc[0, ['x_'+str(j)]] = x_init
        data_store.loc[0, ['y_'+str(j)]] = y_init
        
        # Calculate objective function valuev
        f_init = objfun(x_init, y_init)
        data_store.loc[0, ['f_'+str(j)]] = f_init
        
        # Assign current values of x, y and f to x_best, y_best and f_best
        data_store.loc[0, ['x_best_'+str(j)]] = x_init
        data_store.loc[0, ['y_best_'+str(j)]] = y_init
        data_store.loc[0, ['f_best_'+str(j)]] = f_init    
                
        # Initialise velocities (use low value for slower start)
        data_store.loc[0, ['vx_'+str(j)]] = random.uniform(0, 1)
        data_store.loc[0, ['vy_'+str(j)]] = random.uniform(0, 1)
        
        # Generate random values for rho1 and rho2 ([0,1])
        data_store.loc[0, ['rho1_'+str(j)]] = random.uniform(0, 1)
        data_store.loc[0, ['rho2_'+str(j)]] = random.uniform(0, 1)
        
        # Check for global best and assign x, y and f to global best list when applicable
        if f_init < global_best[2]:
            
            global_best[0] = x_init
            global_best[0] = y_init
            global_best[0] = f_init        
    
    return data_store, global_best

In [41]:
def particle_update(data_store, global_best, c1, c2, w, t, n):
    
    """ 
    This function calculates the generates values for rho1 and rho2velocities, vx(t) and vy(t) 
    associated with x(t-1) and y(t-1) and updates x(t), y(t) and f(t) according to Talbi (2009)
    
    Next, it checks for local and global best solutions and updates if necessary

    The dataframe passed holds as columns (for each particle) the current values of x,y and f, 
    the best values of x,y, and f as well as the velocity and randomly generated rho1 and rho2
    
    Parameters: 
        data_store (DataFrame): Dataframe outlined in the function description
        global_best (list): List of current global best solution in the form [x,y,f]
        c1 (float): Cognitive Factor 1
        c2 (float): Cognitive Factor 2
        w (float): Inertia
        t (int): Current iteration
        n (int): Number of particles

    Returns: 
        data_store (DataFrame): Dataframe outlined in the function description, including velocities
        global_best (list): List of new global best solution in the form [x,y,f]
    """
     


    for i in range(n):
        
        # Generate random values for rho1 and rho2 ([0,1])
        data_store.loc[t-1, ['rho1_'+str(i)]] = random.uniform(0, 1)
        data_store.loc[t-1, ['rho2_'+str(i)]] = random.uniform(0, 1)
    
        # Extract x, y, f, x_best, y_best, vx, vy, rho_1 and rho_2 values from the dataframe
        x = data_store.loc[t-1, ['x_'+str(i)]][0]
        y = data_store.loc[t-1, ['y_'+str(i)]][0]
        f = data_store.loc[t-1, ['f_'+str(i)]][0]
        x_best = data_store.loc[t-1, ['x_best_'+str(i)]][0]
        y_best = data_store.loc[t-1, ['y_best_'+str(i)]][0]
        vx = data_store.loc[t-1, ['vx_'+str(i)]][0]
        vy = data_store.loc[t-1, ['vy_'+str(i)]][0]
        rho1 = data_store.loc[t-1, ['rho1_'+str(i)]][0]
        rho2 = data_store.loc[t-1, ['rho2_'+str(i)]][0]
        
        # Calculate new velocities and update dataframe
        vx_new = w*vx + rho1*c1*(x_best-x) + rho2*c2*(global_best[0] - x)
        vy_new = w*vy + rho1*c1*(y_best-y) + rho2*c2*(global_best[1] - y) 
        data_store.loc[t, ['vx_'+str(i)]] = vx_new
        data_store.loc[t, ['vy_'+str(i)]] = vy_new  
        
        # Calculate new values for x and y and update dataframe
        x_new = x + vx_new
        y_new = y + vy_new
        
        # Check Feasibility. If check_feasibility returns true, keep new solution, else, keep old solution
        if check_feasibility(x_new) == True:
            
            data_store.loc[t, ['x_'+str(i)]] = x_new
            
        else:
            
            x_new = x
            
        if check_feasibility(y_new) == True:
            
            data_store.loc[t, ['y_'+str(i)]] = y_new
            
        else:
            x_new = x
        
        # Calculate new value for f update dataframe
        f_new = objfun(x_new, y_new)
        data_store.loc[t, ['f_'+str(i)]] = f_new
        
        # Check for local best and update if necessary
        if f_new < f:
            
            data_store.loc[t, ['x_best_'+str(i)]] = x_new
            data_store.loc[t, ['y_best_'+str(i)]] = y_new
            data_store.loc[t, ['f_best_'+str(i)]] = f_new
            
        if f_new < global_best[2]:
            
            global_best[0] = x_new
            global_best[1] = y_new
            global_best[2] = f_new   
    
    return data_store, global_best

In [52]:
# Initialise number of particles
n = 50

# Initialise iteration counter, t, the initial swarm locations, velocities, global_best and rho
t = 1
data_store, global_best = dataframe_init(n)

# Declare w, C1 and C2
w = 1
c1 = 1
c2 = 1

# Initialise stop flag and stopping condition counter
stop_flag = False
stop_cond_count = 0
stop_count_max  = 20

while stop_flag == False:
    
    global_best_prev = global_best
    
    data_store, global_best = particle_update(data_store, global_best, c1, c2, w, t, n)
    
    # Stopping condition increment or reset
    if global_best[2] < global_best_prev[2]:
        
        # Reset stopping counter
        stop_cond_count = 0
        
    else: 
        
        # Increment stopping counter
        stop_cond_count = stop_cond_count + 1
   
    # Stopping condition check and set
    if stop_cond_count == stop_count_max:
        
        stop_flag = True
    

In [53]:
global_best

[468.7342422001684, 420.4778185459684, -879.1844682135112]

In [ ]:
velocity_update_rho

In [ ]:
i = 0
t = 1

In [ ]:
data_store, global_best = particle_update(data_store, global_best=[1,2,3], c1=1, c2=1, w=1, t=1, n=2)

In [ ]:
data_store

In [ ]:
a = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

In [ ]:
data_store.loc[len(data_store)] = a

In [ ]:
data_store

In [ ]:
data_store = dataframe_init(2)

In [ ]:
pd.DataFrame(columns = columns_list)

In [ ]:
## Matplotlib Sample Code using 2D arrays via meshgrid
# plt.style.use('dark_background')
# plt.style.use('default')
x = np.arange(-512, 512, 1)
y = np.arange(-512, 512, 1)
x, y = np.meshgrid(x, y)
f = objfuns(x,y)
# figsize=(12,8)
fig = plt.figure(figsize=(12,8))
ax = Axes3D(fig)

surf = ax.plot_surface(x, y, f, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
# ax.set_zlim(-1.01, 1.01)

ax.zaxis.set_major_locator(LinearLocator(10))
ax.set(xlabel='x', ylabel='y', zlabel = 'f')
# ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

fig.colorbar(surf, shrink=0.5, aspect=20)
plt.title('Objective Function Surface',pad=30, fontdict = {'fontsize': 20, 'fontweight' : 5})
plt.show()
fig.savefig('surface_plot.png',dpi=300,bbox_inches = 'tight')